In [83]:
import numpy as np
import pandas as pd
import matplotlib
import anndata as ad
import pickle
import sys

import plotly.io as pio
pio.renderers.default = 'iframe'
import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [5]:
# Read in cleaned version of DF for testing
master_df = pd.read_pickle("./data/clean/cleaned_df_ALL.pi")
master_df

,Patient,sample_name,Indication,Compartment,Archetype,ZBTB5,NPM1P26,NCOR1P3,PRKG2.AS1,MRPS33,...,X.4955,ENSG00000196661,X.13676,SERPINA15P,CDC42SE1,C8orf34,CHAD,ZFPM1.AS1,X_umap1,X_umap2
0,IPIADR001,IPIADR001.T1,ADR,Myeloid,NaN,3.44,17.15,0.0,0.0,22.53,...,0.0,0.00,0.82,0.0,97.07,0.00,0.25,0.0,NaN,NaN
1,IPIADR001,IPIADR001.T1,ADR,Stroma,NaN,7.32,23.63,0.0,0.0,21.62,...,0.0,0.00,2.77,0.0,43.40,0.00,0.70,0.0,NaN,NaN
2,IPIADR001,IPIADR001.T1,ADR,T_cell,NaN,7.29,28.37,0.0,0.0,23.55,...,0.0,0.00,0.73,0.0,139.04,0.00,0.25,0.0,NaN,NaN
3,IPIADR001,IPIADR001.T1,ADR,Tumor,NaN,33.77,8.88,0.0,0.0,47.27,...,0.0,0.00,0.03,0.0,28.49,0.06,0.34,0.0,NaN,NaN
4,IPIADR002,IPIADR002.T1,ADR,Stroma,NaN,2.37,1.64,0.0,0.0,5.26,...,0.0,0.00,3.24,0.0,39.67,0.00,0.00,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,IPISRC040,IPISRC040.T1,SRC,Treg,NaN,5.31,8.95,0.0,0.0,7.11,...,0.0,0.00,25.13,0.0,40.70,0.45,0.00,0.0,NaN,NaN
1516,IPISRC042,IPISRC042.T1,SRC,Myeloid,NaN,1.74,4.35,0.0,0.0,2.91,...,0.0,0.00,0.42,0.0,47.76,0.00,0.00,0.0,NaN,NaN
1517,IPISRC042,IPISRC042.T1,SRC,Stroma,NaN,2.03,0.33,0.0,0.0,13.09,...,0.0,0.00,0.14,0.0,14.84,0.00,0.00,0.0,NaN,NaN
1518,IPISRC042,IPISRC042.T1,SRC,T_cell,NaN,8.99,15.92,0.0,0.0,20.85,...,0.0,0.13,0.34,0.0,78.28,0.00,0.07,0.0,NaN,NaN


In [40]:
non_genes = ["Patient", "sample_name", "Indication", "Compartment", "Archetype", "X_umap1", "X_umap2"]

In [46]:
len(list(set(master_df.columns) - set(non_genes)))

58051

# Create Cleaned Aggregate DF

In [ ]:
#data_path = "./data/230811_Combined_TPM_clean_Tumor_filtered_ALL_T1_T2_INDIVIDUAL_PATIENTS.tsv"
#data_path = "./data/Combined_ALL.tsv"
data_path = "./data/raw/Combined_ALL.tsv"
umap_path = "./data/raw/Pancan_10f_UMAP_COORD.csv"

In [ ]:
umap_df = pd.read_csv(umap_path, sep = ",")
umap_df = umap_df.set_index("Sample")
umap_df = umap_df.rename_axis("sample_name")
data_df = pd.read_csv(data_path, sep = "\t")

In [ ]:
new_cols = ["sample_name", "Compartment", "Patient", "Indication", "Archetype"]
rest_of_cols = list(set(data_df.columns) - set(new_cols))
sorted_df = data_df[new_cols + rest_of_cols]

multiind = sorted_df.set_index(["Patient","sample_name","Indication","Compartment","Archetype"]).sort_index()
merged_df = pd.merge(multiind,umap_df,left_index=True,right_index=True,how="outer")

tidy_df = merged_df.reset_index()

#merged_df.to_pickle("master_df.pi")

# Plotting Tests

## UMAP Example

In [23]:
master_df["NCAM1"]

0        1.56
1       25.53
2        2.65
3       17.65
4       40.78
        ...  
1515    24.29
1516     1.05
1517     0.25
1518     0.67
1519     0.50
Name: NCAM1, Length: 1520, dtype: float64

In [81]:
sum(df == 0)

221

In [78]:
df= master_df["NCAM1"]
#normalized_df=(ncam-ncam.min())/(ncam.max()-ncam.min()) # Min/Max Normalization
normalized_df=(df-df.mean())/df.std() # Mean Normalization

In [102]:
sum(df == 0 )

221

In [111]:
test = np.where(df != 0, np.log2(df),0)

In [112]:
fig = master_df.plot.scatter(x="X_umap1", y = "X_umap2", color = test,)

fig.update_layout(template='simple_white')
fig.update_layout(
    autosize=False,
    width=500,
    height=400,
)

In [13]:
import inspect
inspect.signature(px.scatter)

<Signature (data_frame=None, x=None, y=None, color=None, symbol=None, size=None, hover_name=None, hover_data=None, custom_data=None, text=None, facet_row=None, facet_col=None, facet_col_wrap=0, facet_row_spacing=None, facet_col_spacing=None, error_x=None, error_x_minus=None, error_y=None, error_y_minus=None, animation_frame=None, animation_group=None, category_orders=None, labels=None, orientation=None, color_discrete_sequence=None, color_discrete_map=None, color_continuous_scale=None, range_color=None, color_continuous_midpoint=None, symbol_sequence=None, symbol_map=None, opacity=None, size_max=None, marginal_x=None, marginal_y=None, trendline=None, trendline_options=None, trendline_color_override=None, trendline_scope='trace', log_x=False, log_y=False, range_x=None, range_y=None, render_mode='auto', title=None, template=None, width=None, height=None) -> plotly.graph_objs._figure.Figure>

In [4]:
px.box(master_df, x=["Indication","Compartment"], y="LYVE1")

In [ ]:
fig = master_df[["LYVE1"]].groupby("Indication").plot.box()
fig.show()

# Iso-Seq Stuff

In [ ]:
iso_data = pd.read_csv("/Users/hwismer/Documents/A_01/bcM0001_seurat/bcM0001_collapsed.annotated.info.csv",
                      sep = "\t")

In [ ]:
iso_data["category"]

In [ ]:
iso_data[iso_data["gene"].str.contains("NCAM1")]

In [ ]:
iso_data[iso_data["gene"].str.contains("NCAM1")].groupby("category").size().plot(kind="pie", y = "category",autopct='%.2f')

In [ ]:
iso_data.groupby("category").size().plot(kind="pie", y = "category",autopct='%.2f')

In [ ]:
iso_data[iso_data["gene"].str.contains("IGF2")].groupby("category").size().plot(kind="pie", y = "category",autopct='%.2f')